In [17]:
%load_ext autoreload
%autoreload 2

# Config
import logging
import sys

import arrow
import matplotlib.pyplot as plt
import pandas as pd
import simpy

from src.machine import *
from src.operator import *
from src.utils import *
from src.base import *


logging.basicConfig(
     stream=sys.stdout,
     level=logging.DEBUG,
     format='%(asctime)s - %(name)s - %(levelname)-7s - %(message)s',
     datefmt='%H:%M:%S'
 )
logger = logging.getLogger(__name__)
logger.info('Starting simulation')


# Simulation
start = arrow.now('Europe/Helsinki')
env = simpy.Environment(initial_time=start.timestamp())
machine = Machine(env)
operator = Operator(env).assign_machine(machine)

env.run(until=start.timestamp() + 3 * 24 * 60 * 60)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
03:11:57 - __main__ - INFO    - Starting simulation
03:11:57 - src.base - INFO    - 2022-11-27 03:11:57 - CronBlock(30 8 * * *, 0) - Cron scheduled for 2022-11-27 08:30:00 - 2022-11-27 11:29:59
03:11:57 - src.base - INFO    - 2022-11-27 03:11:57 - CronBlock(30 11 * * *, 0) - Cron scheduled for 2022-11-27 11:30:00 - 2022-11-27 11:59:59
03:11:57 - src.base - INFO    - 2022-11-27 03:11:57 - CronBlock(00 12 * * *, 1) - Cron scheduled for 2022-11-27 12:00:00 - 2022-11-27 13:59:59
03:11:57 - src.base - INFO    - 2022-11-27 03:11:57 - CronBlock(00 14 * * *, 0) - Cron scheduled for 2022-11-27 14:00:00 - 2022-11-27 15:59:59
03:11:57 - src.base - INFO    - 2022-11-27 03:11:57 - Operator - Chilling at home...
03:11:57 - src.base - INFO    - 2022-11-27 08:00:57 - Operator - Working...
03:11:57 - src.base - DEBUG   - 2022-11-27 08:00:57 - Operator - Event - "arrive_at_work"
03:11:57 - src.base - DEBUG   - 2022-1

[autoreload of src.operator failed: Traceback (most recent call last):
  File "/Users/e103089/opt/miniconda3/envs/machine-simulator/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 257, in check
    superreload(m, reload, self.old_objects)
  File "/Users/e103089/opt/miniconda3/envs/machine-simulator/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 480, in superreload
    update_generic(old_obj, new_obj)
  File "/Users/e103089/opt/miniconda3/envs/machine-simulator/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 377, in update_generic
    update(a, b)
  File "/Users/e103089/opt/miniconda3/envs/machine-simulator/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 310, in update_class
    old_obj = getattr(old, key)
  File "/Users/e103089/Documents/Personal/machine-simulator/src/utils.py", line 21, in __get__
    value = getattr(obj, self.private_name)
AttributeError: 'NoneType' object has no attribute '_state'
]

03:11:57 - src.base - DEBUG   - 2022-11-27 11:30:02 - Machine - Event - "switching_off"
03:11:57 - src.base - DEBUG   - 2022-11-27 11:30:02 - Machine - Waiting for production to stop
03:11:57 - src.base - WARNING - 2022-11-27 11:30:02 - Machine - Production interruption already ongoing, returning
03:11:57 - src.base - DEBUG   - 2022-11-27 11:30:06 - Program - Consuming 5.00 of raw-material
03:11:57 - src.base - INFO    - 2022-11-27 11:30:06 - Consumable(raw-material-0) - Container level: 40.00
03:11:57 - src.base - DEBUG   - 2022-11-27 11:30:06 - Program - Event - "program_stopped"
03:11:57 - src.base - DEBUG   - 2022-11-27 11:30:06 - Machine - Event - "production_stopped"
03:11:57 - src.base - DEBUG   - 2022-11-27 11:30:07 - Machine - Event - "switched_off"
03:11:57 - src.base - INFO    - 2022-11-27 11:30:07 - Operator - Having lunch...
03:11:57 - src.base - DEBUG   - 2022-11-27 11:30:07 - Machine - Event - "switching_program_automatically"
03:11:57 - src.base - DEBUG   - 2022-11-27 1

In [19]:
import plotly.express as px


def plot_timeline(data, **kwargs):
    df = (
        pd.DataFrame(data, columns=['ds', 'obj', 'key', 'value'])
        .assign(name=lambda df_: df_['obj'] + ' - ' + df_['key'],
                end_ts=lambda df_: df_.groupby(['obj', 'key'])['ds'].shift(-1),
                value=lambda df_: df_['value'].astype(str))
    )
    if df.empty:
        return

    fig = px.timeline(df, facet_row='name', x_start='ds', x_end='end_ts', y='value',
                      title='Simulation - Categorical', color='obj', **kwargs)
    fig.for_each_annotation(lambda a: a.update(text=a.text.split(' - ')[-1]))
    fig.update_yaxes(matches=None)
    fig.show()


def plot_numerical(data, **kwargs):

    df = pd.DataFrame(data, columns=['ds', 'obj', 'key', 'value'])
    df['end_ts'] = df['ds'].shift(-1)
    if df.empty:
        return

    fig = px.line(df, x='ds', y='value', color='obj', facet_row='key',
                  title='Simulation - Numerical', **kwargs)
    fig.update_yaxes(matches=None)
    fig.for_each_annotation(lambda a: a.update(text=a.text.replace('key=', '')))
    fig.show()


plot_timeline(
    operator.data['categorical']
    + machine.data['categorical']
    + sum([list(p.bom.values())[0]['consumable'].data['categorical']
           for p in machine.programs.values()], [])
    + machine.schedule.data['categorical']
, width=1200, height=1000)

plot_numerical(
    machine.data['numerical']
    + operator.data['numerical']
    + sum([list(p.bom.values())[0]['consumable'].data['numerical']
           for p in machine.programs.values()], [])
,width=800, height=1400)